In [ ]:
# Clean main data table mp_routes

#Import required libraries
import numpy as np
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import pandas_gbq

# Authenticate
auth.authenticate_user()

#Import df, rename Unamed:0 column to route_id, and replace spaces in column titles
query = "SELECT * FROM `rock-finder-project.routes.mp_routes`"
project = "rock-finder-project"
df = pd.read_gbq(query=query, project_id=project).rename(columns={
    'Unnamed: 0': 'route_id', 
    'Avg Stars': 'avg_stars', 
    'Route Type': 'route_type', 
    'Area Latitude': 'area_latitude',
    'Area Longitude': 'area_longitude',
    ' desc':'desc',
    ' protection':'protection'
})

#fill null values in Route column with "Unknown"
df['Route'].fillna("Unknown", inplace=True)

#  Split Location into parts (robust to any number of ' > ' levels)
location_split = df['Location'].str.split(' > ')

#  Extract "area", "region" (island or subregion), and "state" from the last 3 parts
df['state'] = location_split.apply(lambda x: x[-1] if len(x) >= 1 else None)
df['region'] = location_split.apply(lambda x: x[-2] if len(x) >= 2 else None)
df['area'] = location_split.apply(lambda x: x[-3] if len(x) >= 3 else None)
#Fill any nulls in area column with 'Other'
df['area'].fillna('Other', inplace=True)

#  first/topmost part separately:
df['top_level'] = location_split.apply(lambda x: x[0] if len(x) >= 1 else None)

#Clean route_type and create binary columns for 3 relevant route types
core_types = ['Trad', 'Sport', 'TR']
for route in core_types: 
    df[f'is_{route.lower()}'] = df['route_type'].str.contains(rf'\b{route}\b').astype(int)

df[['is_trad', 'is_sport', 'is_tr']].sum()

#Transforms the column Rating into a list. Separates the string elements by " "
df["Rating"] = df["Rating"].str.split()

     

#Create a list with the first element of the split wich is going to contain the rating number and letters 
list_of_first_elements=[]
for l in range(df["Rating"].size):
  list_of_first_elements.append(df["Rating"][l][0])
     

#Create a main rating column  with the list created before
df['main_rating'] = list_of_first_elements
     

#Check for nulls in 'main_rating'
df[df['main_rating'].isnull()]
     

#Get a view of the new column compared to the previous one CAN BE DELEATED 
df[['Rating','main_rating']]
     

#Separating the main rating into number part and string part using a regex finction. this will be saved in 2 new columns. 
#if there are no string values its going to be empty
df[['rating_num', 'rating_cat']] = df['main_rating'].str.extract(r'^(\d+(?:\.\d{1,2})?)(.*)$')
df[['rating_num', 'rating_cat']] #getting a view CAN BE DELETED (LINE ONLY)
     

# Fill null values for rating_num 
df["rating_num"] = df["rating_num"].fillna(df["main_rating"])
     

# TEST rating_num for nulls
df[df["rating_num"].isnull()]
     

# Fill null values for rating_cat
df["rating_cat"] = df["rating_cat"].fillna(df["main_rating"]) #There's still going to be some empty values "". We want that as some of this ratings dont have a category
     

# TEST rating_cat for nulls
df[df["rating_cat"].isnull()]
     

# Get extra information regarding safety of a route from therating column 
     

#Create the rating_safety column
keywords = ["X", "PG13", "R","W"]
df["matches"] = df["Rating"].apply(lambda word_list: [w for w in word_list if w in keywords])
df["rating_safety"] = df["matches"].apply(lambda x: x[0] if x else "safe")
     

#Check for null values in new column rating_safety
df[df["rating_safety"].isnull()]

# Handle invalid "Pitches" and missing "Length"
df['Pitches'] = df['Pitches'].apply(lambda x: abs(x) if x < 0 else x)
df['Length'] = df['Length'].fillna(df['Length'].median())

# Initial cleaning of desc and protection columns
df['desc'] = df['desc'].str.lower()
df['protection'] = df['protection'].str.lower()

df['desc'] = df['desc'].str.strip()
df['protection'] = df['protection'].str.strip()


df['desc'] = df['desc'].str.replace(r'\s+', ' ', regex=True)
df['protection'] = df['protection'].str.replace(r'\s+', ' ', regex=True)

df['desc'] = df['desc'].fillna('')
df['protection'] = df['protection'].fillna('')

import string
df['desc'] = df['desc'].str.translate(str.maketrans('', '', string.punctuation))
df['protection'] = df['protection'].str.translate(str.maketrans('', '', string.punctuation))

#Create df as new table in GBQ
table_id = 'routes.routes_silver'
pandas_gbq.to_gbq(df, table_id, project_id = project, if_exists='replace')